In [71]:
!pip install torchmetrics
!pip install tqdm
!pip install tensorflow


[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python -m pip install --upgrade pip


In [72]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error as  mape
from sklearn.metrics import r2_score
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.manifold import TSNE
from sklearn.neighbors import KNeighborsRegressor as KNR
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR, LinearSVR

from sklearn.linear_model import LinearRegression
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchmetrics import MeanSquaredLogError
from tqdm import tqdm
from torch.optim import AdamW

import tensorflow as tf
from tensorflow import keras as ks
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from keras.wrappers.scikit_learn import KerasClassifier
import seaborn as sns


In [73]:
from preprocess import EstateData

dataset = EstateData("final.csv")

In [74]:
dataset.split_data(stratify=True)

In [75]:
dataset.preprocess(tukey=True)

Start Process!
Fill missing values: Done
Remove outlier: Done
Scale features: Done


In [76]:
train = dataset.train
test = dataset.test

In [77]:
print("train size {}".format(train.shape))
print("test size {}".format(test.shape))

train size (31658, 12)
test size (9278, 12)


In [78]:
train, dev = train_test_split(train, test_size=0.1)

In [79]:
train = pd.get_dummies(train)
dev = pd.get_dummies(dev)
test = pd.get_dummies(test)

In [80]:
features = train.columns.tolist()
#features.remove("Price")
features.remove("PricePerM2")

In [81]:
model = ks.Sequential()
model.add(ks.layers.Dense(100, input_dim = 64, activation = 'relu', name = 'input'))
model.add(ks.layers.Dropout(0.25))
model.add(ks.layers.Dense(100, activation = 'relu', name = 'hidden1'))
model.add(ks.layers.Dropout(0.25))
model.add(ks.layers.Dense(100, activation = 'relu', name = 'hidden2'))
model.add(ks.layers.Dropout(0.2))
model.add(ks.layers.Dense(100, activation = 'relu', name = 'hidden3'))
model.add(ks.layers.Dropout(0.2))
model.add(ks.layers.Dense(1, name = 'output'))
opt = ks.optimizers.Adam(learning_rate = 0.01)
model.compile(optimizer = opt, 
              loss = tf.keras.losses.MeanSquaredLogarithmicError(), 
              metrics = ['mean_absolute_percentage_error'])
train_history = model.fit(train[features], train["PricePerM2"], batch_size = 2048, epochs = 300, validation_split = 0.1)
ann_train_res = model.evaluate(dev[features], dev["PricePerM2"])
ann_test_res = model.evaluate(test[features], test["PricePerM2"])
print("Train:  Loss = %f, Mean_absolute percentage_error = %f" %(ann_train_res[0], ann_train_res[1]/100))
print("Test:  Loss = %f, Mean_absolute_perntage_error = %f" %(ann_test_res[0], ann_test_res[1]/100))

Epoch 1/300
13/13 [==============================] - 2s 38ms/step - loss: 7.8875 - mean_absolute_percentage_error: 346.1795 - val_loss: 3.8266 - val_mean_absolute_percentage_error: 826.7364
Epoch 2/300
13/13 [==============================] - 0s 28ms/step - loss: 2.4982 - mean_absolute_percentage_error: 471.0068 - val_loss: 1.1583 - val_mean_absolute_percentage_error: 121.0770
Epoch 3/300
13/13 [==============================] - 0s 24ms/step - loss: 1.2377 - mean_absolute_percentage_error: 91.4621 - val_loss: 0.9219 - val_mean_absolute_percentage_error: 95.5330
Epoch 4/300
13/13 [==============================] - 0s 24ms/step - loss: 0.9063 - mean_absolute_percentage_error: 119.7608 - val_loss: 0.7789 - val_mean_absolute_percentage_error: 114.6725
Epoch 5/300
13/13 [==============================] - 0s 26ms/step - loss: 0.7559 - mean_absolute_percentage_error: 98.0444 - val_loss: 0.6600 - val_mean_absolute_percentage_error: 84.2205
Epoch 6/300
13/13 [==============================] - 0

In [82]:
model.save('checkpoint/ANN.model')
model = ks.models.load_model('checkpoint/ANN.model')

INFO:tensorflow:Assets written to: checkpoint/ANN.model/assets


INFO:tensorflow:Assets written to: checkpoint/ANN.model/assets


In [83]:
pred = model.predict(test[features])
print("mean_absolute_percentage_error",mape(test["PricePerM2"],pred ))
print("r2_score",r2_score(test["PricePerM2"], pred))



290/290 [==============================] - 1s 2ms/step
mean_absolute_percentage_error 0.702254743428089
r2_score 0.5780781777436454
